In [ ]:
import numpy as np

# Load the NPZ file
data = np.load('unspsc_embeddings.npz')

# List the keys (names) of the arrays in the NPZ file
array_names = data.files

# Create a text file to save the output
with open('unspsc.txt', 'w') as text_file:
    # Access and use the arrays by their names
    for name in array_names:
        array = data[name]
        text_file.write(f'Array name: {name}\n')
        text_file.write(f'Array data:\n{array}\n')


In [ ]:
#check the length of the npz file

import numpy as np

# Load the NPZ file
data = np.load('unspsc_embeddings.npz')
array_names = data.files
num_arrays = len(array_names)
print(f'Number of arrays in the NPZ file: {num_arrays}')



In [ ]:
import pickle

# Open the .pickle file for reading
with open('unspsc_title_list.pickle', 'rb') as file:
    pickle_title = pickle.load(file)

In [ ]:

import re

# Specify unspsc text file
text_file_path = 'unspsc.txt'


array_names = []
array_data = []

# Open the text file and process its contents
with open(text_file_path, 'r') as file:
    lines = file.readlines()
    

    # Initialize variables to capture Array_name and Array_data
    current_name = None
    current_data = []

    # Iterate through lines in the file
    for line in lines:
        #print(line)
        # Check if the line contains Array_name
        name_match = re.match(r'Array\s*name\s*:\s*(\w+)', line)
        #print(name_match)
        if name_match:
            # If we already have data from a previous Array_data section, append it
            if current_name is not None:
                array_names.append(current_name)
                array_data.append(current_data)
            
            # Set the current Array_name and reset data
            current_name = name_match.group(1)
            current_data = []
        elif current_name is not None:
            # Check if the line contains numeric data
            data_match = re.findall(r'([-+]?\d*\.\d+e[+-]?\d+|\d+)', line)
            if data_match:
                # Convert scientific notation strings to floating-point numbers
                current_data.extend([float(num) for num in data_match])

    # Append the last set of data if any
    if current_name is not None:
        array_names.append(current_name)
        array_data.append(current_data)

# Create a DataFrame
df_unspsc = pd.DataFrame({'Array_name': array_names, 'Array_data': array_data})

In [ ]:
#Rename the Array_name to UNSPSC titles
df_unspsc['Array_name'] = pickle_title[:len(df_unspsc)]

In [ ]:
df_unspsc = df_unspsc.rename(columns={'Array_name': 'UNSPSC Title'})

In [ ]:
type(df_unspsc['Array_data'][0])

In [ ]:
#this file contains the tender details with Descrption embeddings
df_tenders = pd.read_excel('merged_file.xlsx')


In [ ]:
#change the embeddings from str to list
import ast
df_tenders['Embeddings'] = df_tenders['Embeddings'].apply(ast.literal_eval)

In [ ]:
#from df_tenders dropping the duplicate value
reference_column_name = "UNSPSC Title"
unique_references_df = df_tenders.drop_duplicates(subset=reference_column_name, keep='first')
unique_references_df = unique_references_df.reset_index(drop=True)

In [ ]:
#joining the two data frames by "UNSPSC Title"
merged_df = unique_references_df.merge(df_unspsc, on='UNSPSC Title', how='inner')

In [ ]:
#creating a new data frame with below columns.
selected_columns = ['Reference Number', 'UNSPSC Title', 'Embeddings','Array_data']
filter_df = merged_df[selected_columns]

2 of the tenders description embeddings has more than 768 embeddings removing 2 rows.

In [ ]:
desired_size = 768  # Change this to your desired size (e.g., 768)
filter_df = filter_df[filter_df['Embeddings'].apply(lambda x: len(x) == desired_size)]

In [ ]:
filter_df

In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
# Calculate cosine similarity for each pair of embeddings
cosine_similarities = []
for index, row in filter_df.iterrows():
    embedding1 = row['Embeddings']
    embedding2 = row['Array_data']
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    cosine_similarities.append(similarity)

# Add the cosine similarities as a new column in the DataFrame
filter_df['Cosine_Similarity'] = cosine_similarities



In [ ]:
threshold_val = -0.05  # Replace with your desired threshold value

# Filter rows based on 'Cosine_Similarity' column
prblm_embedding = filter_df[filter_df['Cosine_Similarity'] < threshold_val]

In [ ]:
prblm_embedding

In [ ]:
# Specify the file path where you want to save the Excel file
excel_file_path = 'unspsc_similarity.xlsx'

# Save the DataFrame to an Excel file
filter_df.to_excel(excel_file_path, index=False)